# Dockerfile

Dockerfile 是用來描述 container image 內容該怎麼建立

<img src="./img/Dockerfile.png" style="width:360px"></img>


In [1]:
# 一個簡單的 nodejs 網頁程式
# 當有 http request 時候會列印 os的hostname
# listen port 爲 8080
! cat hostname/app.js

const http = require('http');
const os = require('os');

console.log("Kubia server starting...");

var handler = function(request, response) {
  console.log("Received request from " + request.connection.remoteAddress);
  response.writeHead(200);
  response.end("You've hit " + os.hostname() + "\n");
};

var www = http.createServer(handler);
www.listen(8080);


In [2]:
%%bash --bg --proc bg_proc
# 第一行指令 是要告訴Jupyterlab 
# 要在背景執行下面的 nodejs 指令 
nodejs hostname/app.js

In [3]:
# 利用 curl access localhost:8080
! curl localhost:8080

You've hit b95a58b02288


In [4]:
# terminal the node application
! echo "ubuntu" | sudo -S pkill node

[sudo] password for ubuntu: 

In [5]:
# 查詢這個 nodejs 的 pid
! ps aux | grep 'hostname/app.js' | grep -v 'grep' 

In [6]:
# kill 掉這隻 nodejs 的 pid
# ! ps aux | grep 'hostname/app.js' | grep -v 'grep' | awk '{print $2}' | xargs kill

In [7]:
# 檢查
! ps aux | grep 'hostname/app.js' | grep -v 'grep' 

In [8]:
# Dockerfile是用來描述如何建立docker image
! cat hostname/Dockerfile

FROM node:7
ADD app.js /app.js
ENTRYPOINT ["node", "app.js"]


- `FROM` 指定base image, `node:7`，第7版的nodejs 
- `ADD` 把檔案加到Docker image裏面, hostname/app.js 複製到 contianer 的 /app.js
- `ENTRYPOINT` 是指定 container 初始化時執行的指令

In [9]:
# 你需要在terminal輸入 docker login 
# 登入你docker hub 賬戶
# 按左邊的 Filer Browser (Ctrl+Shift+F)
# 再按上面的藍色 New Launcher
! docker login

Authenticating with existing credentials...
WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [10]:
# docker build -t <your-docker-user-name>/image-name:tag Dockerfile-Path
# docker build -t <your-docker-user-name>/hostname:v1 hostname
! docker build -t brokenpen/hostname:v1 hostname

Sending build context to Docker daemon  3.072kB
Step 1/3 : FROM node:7
 ---> d9aed20b68a4
Step 2/3 : ADD app.js /app.js
 ---> Using cache
 ---> c940864095a2
Step 3/3 : ENTRYPOINT ["node", "app.js"]
 ---> Using cache
 ---> ddd5107763d6
Successfully built ddd5107763d6
Successfully tagged brokenpen/hostname:v1


In [11]:
# pid=$(docker run -d -p 8080:8080 <your_docker_username>/hostname:v1)
! docker rm -f app
! docker run -d -p 8080:8080 --name app brokenpen/hostname:v1

app
95244f5f9820a0a1648320778b5364d88ec1b2d30691374b508d49c00a3392ec


In [12]:
# http request to localhost:8080
! curl localhost:8080

curl: (56) Recv failure: Connection reset by peer


In [13]:
# 強制停止並且移除這個app 的container
! docker rm -f app

app


In [14]:
# 或是先停止這給app container
! docker stop app

Error response from daemon: No such container: app


In [15]:
# 再把這個app container移除
! docker rm app

Error: No such container: app
